In [ ]:
%env WORKDIR=/tmp/vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

# Using ACME

In [ ]:
! vault read pki_int/config/cluster

In [ ]:
%%bash
vault secrets tune \
      -passthrough-request-headers=If-Modified-Since \
      -allowed-response-headers=Last-Modified \
      -allowed-response-headers=Location \
      -allowed-response-headers=Replay-Nonce \
      -allowed-response-headers=Link \
      pki_int

In [ ]:
! vault write pki_int/config/acme enabled=true

In [ ]:
%%bash
# Configure ACME directory for the PKI role
vault write pki_int/roles/example-dot-com-acme \
    allowed_domains="*"\
    allow_subdomains=true \
    allow_bare_domains=false \
    allow_localhost=true \
    max_ttl=720h \
    ttl=60s \
    require_cn=false

In [ ]:
%%bash
# Verify ACME is enabled
vault read pki_int/config/acme

# Deploy Caddy Web Server with Vault ACME

Based on: https://developer.hashicorp.com/vault/tutorials/pki/pki-acme-caddy

Caddy will automatically obtain certificates from Vault via ACME protocol.

## Step 1: Create Namespace

In [ ]:
%%bash
kubectl create namespace caddy-acme --dry-run=client -o yaml | kubectl apply -f -
kubectl get namespace caddy-acme

## Step 2: Create Caddyfile ConfigMap

This Caddyfile configures Caddy to use Vault's ACME endpoint for automatic HTTPS.

In [ ]:
%%bash
cat > $WORKDIR/caddy-config.yaml <<'EOF'
apiVersion: v1
kind: ConfigMap
metadata:
  name: caddy-config
  namespace: caddy-acme
data:
  Caddyfile: |
    {
      # Configure ACME to use Vault
      acme_ca https://vault.vault.svc.cluster.local:8200/v1/pki_int/acme/directory
      # Use insecure for self-signed Vault CA (for demo only!)
      acme_ca_root /vault-ca/ca.crt
    }
    
    # Your domain
    caddy.caddy-acme.svc.cluster.local {
      respond "Hello from Caddy with Vault ACME! 🎉"
      
      # Show certificate info
      header Server "Caddy with Vault ACME"
      
      log {
        output stdout
        format console
      }
    }
EOF

kubectl apply -f $WORKDIR/caddy-config.yaml
kubectl get configmap -n caddy-acme

## Step 3: Create Vault CA Secret

Caddy needs to trust Vault's CA certificate.

In [ ]:
%%bash
kubectl create secret generic vault-ca \
  --from-file=ca.crt=$VAULT_CACERT \
  -n caddy-acme \
  --dry-run=client -o yaml | kubectl apply -f -

kubectl get secret vault-ca -n caddy-acme

## Step 4: Deploy Caddy Pod

Caddy will automatically request a certificate from Vault via ACME.

In [ ]:
%%bash
cat > $WORKDIR/caddy-deployment.yaml <<'EOF'
apiVersion: apps/v1
kind: Deployment
metadata:
  name: caddy
  namespace: caddy-acme
  labels:
    app: caddy
spec:
  replicas: 1
  selector:
    matchLabels:
      app: caddy
  template:
    metadata:
      labels:
        app: caddy
    spec:
      containers:
      - name: caddy
        image: caddy:2-alpine
        ports:
        - containerPort: 80
          name: http
        - containerPort: 443
          name: https
        volumeMounts:
        - name: config
          mountPath: /etc/caddy
          readOnly: true
        - name: vault-ca
          mountPath: /vault-ca
          readOnly: true
        - name: caddy-data
          mountPath: /data
        - name: caddy-config-cache
          mountPath: /config
        env:
        - name: CADDY_ACME_EMAIL
          value: "admin@caddy-acme.svc.cluster.local"
      volumes:
      - name: config
        configMap:
          name: caddy-config
      - name: vault-ca
        secret:
          secretName: vault-ca
      - name: caddy-data
        emptyDir: {}
      - name: caddy-config-cache
        emptyDir: {}
---
apiVersion: v1
kind: Service
metadata:
  name: caddy
  namespace: caddy-acme
spec:
  selector:
    app: caddy
  ports:
  - name: http
    port: 80
    targetPort: 80
  - name: https
    port: 443
    targetPort: 443
  type: ClusterIP
EOF

kubectl apply -f $WORKDIR/caddy-deployment.yaml

echo ""
echo "Waiting for Caddy to start..."
sleep 5

kubectl get pods,svc -n caddy-acme

## Step 5: Check Caddy Logs

Watch Caddy obtain a certificate from Vault via ACME.

In [ ]:
%%bash
echo "=== Caddy Pod Status ==="
kubectl get pods -n caddy-acme

echo ""
echo "=== Caddy Logs (last 50 lines) ==="
POD=$(kubectl get pods -n caddy-acme -l app=caddy -o jsonpath='{.items[0].metadata.name}')
kubectl logs -n caddy-acme $POD --tail=50 | grep -E "(ACME|certificate|error|https)" || kubectl logs -n caddy-acme $POD --tail=50

### Monitor Certificate Acquisition

Watch the logs to see Caddy successfully obtain a certificate from Vault.

In [ ]:
%%bash
echo "=== Waiting for Caddy to start... ==="
sleep 5

echo ""
echo "=== Caddy Logs - Watch for ACME certificate acquisition ==="
POD=$(kubectl get pods -n caddy-acme -l app=caddy -o jsonpath='{.items[0].metadata.name}')
kubectl logs -n caddy-acme $POD --tail=100 | grep -E "(obtain|ACME|certificate|valid|challenge|success)" || kubectl logs -n caddy-acme $POD --tail=30

### Test HTTPS with Service DNS Name

Now test the HTTPS endpoint using the service DNS name.

In [ ]:
%%bash
echo "=== Test HTTPS endpoint (using service DNS name) ==="
kubectl run curl-https-test --image=curlimages/curl --rm -it --restart=Never -- \
  curl -k -v https://caddy.caddy-acme.svc.cluster.local/ 2>&1 | grep -E "(HTTP|Hello|Server|subject|issuer|CN=)" | head -20

echo ""
echo ""
echo "=== View Certificate Details ==="
kubectl run openssl-test --image=alpine/openssl --rm -it --restart=Never -- \
  s_client -connect caddy.caddy-acme.svc.cluster.local:443 -servername caddy.caddy-acme.svc.cluster.local <<< "Q" 2>/dev/null | \
  grep -A 2 "subject\|issuer" | head -10

## Step 6: Test Caddy with ACME Certificate

Test the HTTPS endpoint secured by Vault-issued certificate.

In [ ]:
%%bash
echo "=== Testing HTTP (should redirect to HTTPS) ==="
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -- \
  curl -v http://caddy.caddy-acme.svc.cluster.local/ 2>&1 | head -20

echo ""
echo ""
echo "=== Testing HTTPS with Vault CA ==="
# Copy Vault CA into a test pod and use it
kubectl run -it --rm curl-https-test --image=curlimages/curl --restart=Never -- \
  sh -c "curl -k -v https://caddy.caddy-acme.svc.cluster.local/ 2>&1" | grep -E "(HTTP|Hello|Server|subject|issuer)" || \
  kubectl run -it --rm curl-https-test --image=curlimages/curl --restart=Never -- \
  curl -k https://caddy.caddy-acme.svc.cluster.local/

echo ""

### Understanding the Caddy ACME Flow

When Caddy starts, it automatically:

1. **Reads Caddyfile** - Sees `caddy.caddy-acme.svc.cluster.local` needs HTTPS
2. **Contacts Vault ACME** - Registers account at `https://vault.vault.svc.cluster.local:8200/v1/pki_int/acme/directory`
3. **Requests Certificate** - For `caddy.caddy-acme.svc.cluster.local`
4. **HTTP-01 Challenge** - Vault sends challenge token
5. **Serves Challenge** - Caddy serves token at `/.well-known/acme-challenge/TOKEN`
6. **Vault Validates** - Vault checks HTTP endpoint for token (requires DNS/connectivity to pod)
7. **Certificate Issued** - Vault signs certificate, Caddy saves to `/data/caddy`
8. **HTTPS Enabled** - Caddy serves on port 443 with Vault-issued certificate

**Key Difference from cert-manager:**
- **Caddy**: Application handles ACME directly, manages own certificates
- **cert-manager**: External controller requests certificates, injects as Kubernetes Secrets

**Note**: For HTTP-01 to work, `caddy.caddy-acme.svc.cluster.local` must resolve to the Caddy pod. In this cluster environment, service DNS resolution handles this automatically.

# ACME Cert Manager

## cert-manager with Vault ACME - Using Internal DNS

For cert-manager to work with Vault ACME in a cluster, the Certificate must request a domain that:
1. **Vault can resolve** via Kubernetes DNS (CoreDNS)
2. **Vault can reach** over the cluster network for HTTP-01 validation

We'll use the **ingress controller's service DNS name** as the certificate domain.

### Step 1: Create Vault ACME Role for Cluster Domains

First, ensure Vault ACME role allows cluster-local domains.

In [ ]:
%%bash
# Create or update ACME role to allow cluster-local domains
vault write pki_int/roles/acme-cluster-local \
    allowed_domains="*.local,*.cluster.local,*.svc.cluster.local,localhost" \
    allow_subdomains=true \
    allow_bare_domains=true \
    allow_localhost=true \
    allow_ip_sans=true \
    ttl=60s \
    max_ttl=720h \
    require_cn=false

echo ""
echo "✅ ACME role allows internal cluster domains"
vault read pki_int/roles/acme-cluster-local

### Step 2: Create ClusterIssuer with Vault ACME

Configure cert-manager to use Vault as the ACME server.

In [ ]:
%%bash
# Create ClusterIssuer for Vault ACME
VAULT_CA_B64=$(cat $VAULT_CACERT | base64 | tr -d '\n')

cat > $WORKDIR/cluster-issuer-internal.yaml <<EOF
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: vault-acme-internal
spec:
  acme:
    server: https://vault.vault.svc.cluster.local:8200/v1/pki_int/acme/directory
    caBundle: ${VAULT_CA_B64}
    privateKeySecretRef:
      name: vault-acme-account-key
    solvers:
    - http01:
        ingress:
          ingressClassName: nginx
EOF

kubectl apply -f $WORKDIR/cluster-issuer-internal.yaml

echo ""
echo "Checking ClusterIssuer status..."
sleep 3
kubectl get clusterissuer vault-acme-internal
kubectl describe clusterissuer vault-acme-internal | grep -A 5 "Status:"

### Step 3: Deploy Test Application with Ingress

Create a simple app and ingress using an **internal FQDN** that Vault can reach for HTTP-01 validation.

In [ ]:
%%bash
# Create namespace for test app
kubectl create namespace acme-test --dry-run=client -o yaml | kubectl apply -f -

# Deploy simple test application
cat > $WORKDIR/test-app.yaml <<'EOF'
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-app
  namespace: acme-test
spec:
  replicas: 1
  selector:
    matchLabels:
      app: hello
  template:
    metadata:
      labels:
        app: hello
    spec:
      containers:
      - name: hello
        image: hashicorp/http-echo
        args:
        - "-text=ACME Test Application with cert-manager and Vault"
        ports:
        - containerPort: 5678
---
apiVersion: v1
kind: Service
metadata:
  name: hello-service
  namespace: acme-test
spec:
  selector:
    app: hello
  ports:
  - port: 80
    targetPort: 5678
EOF

kubectl apply -f $WORKDIR/test-app.yaml

echo ""
echo "✅ Test application deployed"
kubectl get pods,svc -n acme-test

### Step 4: Create Ingress with Ingress Controller's DNS Name

**Critical:** The ingress hostname must be a **real, resolvable DNS name** that Vault can reach.

We use the **ingress controller's own service DNS name** as the hostname:
- Format: `<ingress-service-name>.<namespace>.svc.cluster.local`
- Example: `ingress-nginx-controller.ingress-nginx.svc.cluster.local`

**Why this works:**
- ✅ This DNS name actually exists (it's the ingress controller's service)
- ✅ Vault can resolve it via CoreDNS
- ✅ Vault can reach it for HTTP-01 validation
- ✅ The ingress controller routes to the backend based on the Host header

In [ ]:
%%bash
# Get the ingress controller service name and namespace
INGRESS_SVC=$(kubectl get svc -n ingress-nginx -o jsonpath='{.items[0].metadata.name}' 2>/dev/null)
INGRESS_NS="ingress-nginx"

if [ -z "$INGRESS_SVC" ]; then
    echo "⚠️  No ingress controller found. Install ingress-nginx first:"
    echo "   kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/controller-v1.8.1/deploy/static/provider/cloud/deploy.yaml"
    exit 1
fi

# Build the ingress controller's internal FQDN
INGRESS_FQDN="${INGRESS_SVC}.${INGRESS_NS}.svc.cluster.local"

echo "Found ingress controller: $INGRESS_SVC"
echo "Using DNS name: $INGRESS_FQDN"

# Create ingress using the ingress controller's DNS name as hostname
cat > $WORKDIR/ingress-internal.yaml <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: hello-ingress
  namespace: acme-test
  annotations:
    cert-manager.io/cluster-issuer: vault-acme-internal
    nginx.ingress.kubernetes.io/ssl-redirect: "false"
spec:
  ingressClassName: nginx
  tls:
  - hosts:
    - ${INGRESS_FQDN}
    secretName: hello-tls-cert
  rules:
  - host: ${INGRESS_FQDN}
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: hello-service
            port:
              number: 80
EOF

kubectl apply -f $WORKDIR/ingress-internal.yaml

echo ""
echo "✅ Ingress created with DNS name: ${INGRESS_FQDN}"
echo "   This DNS name resolves to the ingress controller's ClusterIP"
echo ""
echo "Waiting for cert-manager to request certificate..."
sleep 5
kubectl get ingress -n acme-test

### Step 5: Monitor Certificate Request

Watch cert-manager request the certificate from Vault via ACME. This may take 30-60 seconds.

In [ ]:
%%bash
echo "=== Certificate Resource Status ==="
kubectl get certificate -n acme-test

echo ""
echo "=== Certificate Details ==="
kubectl describe certificate -n acme-test hello-tls-cert | grep -A 10 "Status:"

echo ""
echo "=== CertificateRequest Status ==="
kubectl get certificaterequest -n acme-test

echo ""
echo "=== Challenge Status (if any) ==="
kubectl get challenges -n acme-test 2>/dev/null || echo "No active challenges"

echo ""
echo "=== Secret Created (certificate) ==="
kubectl get secret -n acme-test hello-tls-cert 2>/dev/null || echo "Certificate secret not yet created"

### Understanding the HTTP-01 Flow with Internal DNS

**What happens:**

1. **Ingress Created** → cert-manager sees `cert-manager.io/cluster-issuer` annotation
2. **Certificate Resource Created** → cert-manager creates Certificate for ingress controller's DNS name (e.g., `ingress-nginx-controller.ingress-nginx.svc.cluster.local`)
3. **ACME Registration** → cert-manager registers with Vault ACME endpoint
4. **HTTP-01 Challenge Initiated** → Vault sends challenge token
5. **Challenge Ingress Created** → cert-manager creates temporary ingress at `/.well-known/acme-challenge/TOKEN`
6. **Vault Validates** → Vault makes HTTP request to `http://ingress-nginx-controller.ingress-nginx.svc.cluster.local/.well-known/acme-challenge/TOKEN`
   - ✅ **DNS Resolution:** CoreDNS resolves the ingress controller's service DNS name to its ClusterIP
   - ✅ **Network Access:** Vault can reach ingress controller over cluster network
   - ✅ **Challenge Response:** Ingress controller serves the challenge token
7. **Certificate Issued** → Vault signs certificate and returns it
8. **Secret Created** → cert-manager stores certificate in `hello-tls-cert` Secret
9. **Ingress Updated** → Ingress uses the certificate for TLS

**Why this works:**
- We use the **ingress controller's actual service DNS name** as the certificate hostname
- This DNS name resolves internally via CoreDNS to the ingress controller's ClusterIP
- Vault (inside cluster) can reach the ingress controller directly
- The ingress controller routes traffic based on the `Host` header to the correct backend service
- No external DNS or internet access required!

**Key Insight:**
The ingress hostname must be a **real, resolvable DNS name** from Vault's perspective. Using the ingress controller's own service DNS name ensures this.

### Step 6: Test the Application with HTTPS

Once the certificate is issued, test the application via the ingress.

In [ ]:
%%bash
# Get the ingress controller's DNS name
INGRESS_SVC=$(kubectl get svc -n ingress-nginx -o jsonpath='{.items[0].metadata.name}')
INGRESS_FQDN="${INGRESS_SVC}.ingress-nginx.svc.cluster.local"

echo "Testing application via ingress: $INGRESS_FQDN"
echo ""

# First, verify certificate was issued
echo "=== Certificate Status ==="
kubectl get certificate -n acme-test
kubectl get secret -n acme-test hello-tls-cert 2>/dev/null && echo "✅ Certificate secret exists" || echo "❌ Certificate not yet issued"

echo ""
echo ""
echo "=== Test HTTPS Endpoint ==="
kubectl run test-https --image=curlimages/curl --rm -it --restart=Never -- \
  curl -k -v https://${INGRESS_FQDN}/ -H "Host: ${INGRESS_FQDN}" 2>&1 | grep -E "HTTP|Hello|Server|subject|issuer" | head -15


#### Verify Certificate Validity Period

Extract and display the certificate's notBefore, notAfter dates and calculate the duration.

In [ ]:
%%bash
# Get the ingress controller's DNS name
INGRESS_SVC=$(kubectl get svc -n ingress-nginx -o jsonpath='{.items[0].metadata.name}')
INGRESS_FQDN="${INGRESS_SVC}.ingress-nginx.svc.cluster.local"

echo "📅 Certificate Validity Information"
echo "===================================="
echo ""
echo "Fetching certificate from: https://${INGRESS_FQDN}"
echo ""

# Use openssl s_client to get the certificate and extract validity dates
kubectl run cert-info --image=nicolaka/netshoot --rm -it --restart=Never -- \
  bash -c "
    # Get certificate via openssl
    CERT=\$(echo | openssl s_client -connect ${INGRESS_FQDN}:443 -servername ${INGRESS_FQDN} 2>/dev/null | openssl x509 -noout -dates)
    
    echo '📋 Certificate Validity Dates:'
    echo '------------------------------'
    echo \"\$CERT\"
    echo ''
    
    # Extract notBefore and notAfter
    NOT_BEFORE=\$(echo \"\$CERT\" | grep notBefore | cut -d'=' -f2)
    NOT_AFTER=\$(echo \"\$CERT\" | grep notAfter | cut -d'=' -f2)
    
    echo '🕐 Formatted Dates:'
    echo '-------------------'
    echo \"  Start (notBefore): \$NOT_BEFORE\"
    echo \"  End   (notAfter):  \$NOT_AFTER\"
    echo ''
    
    # Calculate duration in seconds
    START_EPOCH=\$(date -d \"\$NOT_BEFORE\" +%s 2>/dev/null || date -j -f '%b %d %T %Y %Z' \"\$NOT_BEFORE\" +%s 2>/dev/null)
    END_EPOCH=\$(date -d \"\$NOT_AFTER\" +%s 2>/dev/null || date -j -f '%b %d %T %Y %Z' \"\$NOT_AFTER\" +%s 2>/dev/null)
    
    if [ ! -z \"\$START_EPOCH\" ] && [ ! -z \"\$END_EPOCH\" ]; then
        DURATION_SECONDS=\$((END_EPOCH - START_EPOCH))
        DURATION_MINUTES=\$((DURATION_SECONDS / 60))
        DURATION_HOURS=\$((DURATION_SECONDS / 3600))
        DURATION_DAYS=\$((DURATION_SECONDS / 86400))
        
        echo '⏱️  Certificate Duration:'
        echo '------------------------'
        echo \"  Total: \$DURATION_SECONDS seconds\"
        echo \"         \$DURATION_MINUTES minutes\"
        echo \"         \$DURATION_HOURS hours\"
        echo \"         \$DURATION_DAYS days\"
        echo ''
        
        # Calculate remaining time
        NOW_EPOCH=\$(date +%s)
        REMAINING_SECONDS=\$((END_EPOCH - NOW_EPOCH))
        
        if [ \$REMAINING_SECONDS -gt 0 ]; then
            REMAINING_MINUTES=\$((REMAINING_SECONDS / 60))
            echo '⏳ Time Remaining:'
            echo '------------------'
            echo \"  \$REMAINING_SECONDS seconds\"
            echo \"  \$REMAINING_MINUTES minutes\"
            
            if [ \$REMAINING_SECONDS -lt 120 ]; then
                echo ''
                echo '⚠️  Certificate expires in less than 2 minutes!'
                echo '   cert-manager should renew soon (at 2/3 of TTL)'
            fi
        else
            echo '❌ Certificate has EXPIRED!'
        fi
    fi
    
    echo ''
    echo '🔍 Full Certificate Details:'
    echo '----------------------------'
    echo | openssl s_client -connect ${INGRESS_FQDN}:443 -servername ${INGRESS_FQDN} 2>/dev/null | \
      openssl x509 -noout -text | grep -A 2 'Validity'
    
    echo ''
    echo '🔐 Certificate Serial and Issuer:'
    echo '----------------------------------'
    echo | openssl s_client -connect ${INGRESS_FQDN}:443 -servername ${INGRESS_FQDN} 2>/dev/null | \
      openssl x509 -noout -serial -issuer
  "

echo ""
echo "✅ Certificate information retrieved successfully"